In [ ]:
# Install necessary libraries
%pip install pandas

import pandas as pd

# Load dataset from CSV file

df = pd.read_csv("/content/Tweets.csv")

# Show a sample of the dataset
print(df[['text', 'airline_sentiment']].head())

In [ ]:
# Install necessary libraries
!pip install nltk

import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

# Preprocessing function
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^A-Za-z\s]', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    return ' '.join(tokens)

# Apply preprocessing to the dataset
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Show cleaned text
print(df[['text', 'cleaned_text']].head())

In [ ]:
# Install necessary libraries
!pip install scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization
tfidf = TfidfVectorizer(max_features=5000)  # Limit to top 5000 features
X = tfidf.fit_transform(df['cleaned_text']).toarray()

# Labels (convert sentiments to numerical values)
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['airline_sentiment'])

print(X.shape)  # Check the feature dimensions


In [ ]:
# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)


In [ ]:
!pip install tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define a simple feed-forward neural network
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dense(64, activation='relu'))
nn_model.add(Dense(3, activation='softmax'))  # 3 output classes: Positive, Negative, Neutral

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
nn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


In [ ]:
# Naive Bayes Evaluation
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

# Neural Network Evaluation
nn_pred = nn_model.predict(X_test)
nn_pred_labels = nn_pred.argmax(axis=1)
print(f"Accuracy: {accuracy_score(y_test, nn_pred_labels)}")
print(classification_report(y_test, nn_pred_labels, target_names=encoder.classes_))

In [ ]:
import pickle

# Save the trained model
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

# Save the TF-IDF vectorizer
with open('tfidf.pkl', 'wb') as tfidf_file:
    pickle.dump(tfidf, tfidf_file)


In [ ]:
# Save the Keras model
nn_model.save('nn_model.h5')  # Save the entire model

In [ ]:
!pip install flask flask-ngrok


In [ ]:
from flask import Flask, request, jsonify, render_template_string
from flask_ngrok import run_with_ngrok
import pickle
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# Download NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Use ngrok to run the app

# Load your model and TF-IDF vectorizer (make sure to save your model and vectorizer before this)
# For this example, I'm assuming you've already trained and saved your model and TF-IDF.
# Replace 'model.pkl' and 'tfidf.pkl' with the paths to your saved files.
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('tfidf.pkl', 'rb') as f:
    tfidf = pickle.load(f)

# Preprocessing function
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters, numbers, and punctuations
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# HTML template for the web interface
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis</title>
</head>
<body>
    <h1>Sentiment Analysis for Social Media Posts</h1>
    <form id="form" action="#" method="POST">
        <textarea name="text" placeholder="Enter social media post here" required></textarea>
        <br>
        <button type="submit">Analyze Sentiment</button>
    </form>
    <h2 id="result"></h2>
    <script>
        document.getElementById('form').onsubmit = async (e) => {
            e.preventDefault();
            const formData = new FormData(e.target);
            const response = await fetch('/predict', {
                method: 'POST',
                body: JSON.stringify({text: formData.get('text')}),
                headers: {'Content-Type': 'application/json'}
            });
            const result = await response.json();
            document.getElementById('result').innerText = 'Sentiment: ' + result.sentiment;
        };
    </script>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route('/predict', methods=['POST'])
def predict():
    input_text = request.json['text']
    processed_text = preprocess_text(input_text)
    text_vector = tfidf.transform([processed_text])
    prediction = model.predict(text_vector)
    sentiment = prediction[0]  # Assuming you have already encoded your labels
    return jsonify({'sentiment': sentiment})

# Run the app
if __name__ == '__main__':
    app.run()
